In [1]:
from ipywidgets import widgets
from IPython.display import display,clear_output
from comfortCalc import calcComf
from riskCalc import calcRisk
from StreetGraph import Graph,Node,Edge
from save import saveGraph
from generateconf import generateConf
from interactiveMap import GUI
from tkinter import *
import ast,os,pickle

class main(object):
    def __init__(self):
        self.strfile,self.rskfile,self.usecmf = '','',''
        self.cmffile,self.cmffnc,self.buildG = '','',''
        self.grphfile,self.mpfile,self.mpcoords = '','',''
        self.path = os.path.dirname(os.getcwd()) # Path to output file
    
    def defStreets(self,Street_file): self.strfile = Street_file
    def defRisks(self,Risk_file): self.rskfile = Risk_file
    def useComfFunct(self,useComfnc): self.usecmf = useComfnc
    def defComfort(self,Discomfort_file): self.cmffile = Discomfort_file
    #def defComfortFnc(self,Discomfort_function): self.cmffnc = Discomfort_function
    def graphBuild(self,Build_graph): self.buildG = Build_graph
    def defGraph(self,Graph_file): self.grphfile = Graph_file
    def defMap(self,Map_file): self.mpfile = Map_file
    def defMcoord(self,Map_coordinates): self.mpcoords = ast.literal_eval(Map_coordinates)
    
    def loadGraph(self):
        savefile = open(self.path + self.grphfile, 'rb')
        self.graph = pickle.load(savefile)
    
    def buildGraph(self):
        ## Build graph ##
        graph = Graph()
        
        f = open(self.path+self.rskfile) # extract risk
        f.readline()
        riskDict = {}
        for line in f:
            line = line.split(',')
            Id = line[0]
            rsk = float(line[4][:-1])
            riskDict[Id] = rsk
        f.close()
        
        if self.usecmf == 'no':
            print('Not implemented, yet. The comfort function will be used instead')
        
        f = open(self.path+self.strfile,'r') # add Nodes
        f.readline()
        for line in f: 
            line = line.split('\t')
            Id = line[0]
            coords = ast.literal_eval(line[1]);
            lat = coords[0]; lon = coords[1]; height = coords[2]
            try: risk = riskDict[Id] # check whether there is a risk defined for this node
            except KeyError: continue
            graph.addNode(Node(Id,lat,lon,height,risk))
        f.close()
        
        f = open(self.path+self.strfile,'r') # add Edges
        f.readline()
        for line in f: 
            line = line.split('\t')
            Id = line[0]
            tgtNodes = ast.literal_eval(line[2][:-1])
            try: src = graph.nodes[list(map(str,graph.nodes)).index(Id)]
            except ValueError: continue
            for el in tgtNodes:
                try: dest = graph.nodes[list(map(str,graph.nodes)).index(el)]
                except ValueError: continue
                edge = Edge(src,dest)
                edge.setWeight(calcRisk(edge),calcComf(edge))
                graph.addEdge(edge)
        f.close()
        
        saveGraph(graph,self.grphfile)
        self.graph = graph
        ## Graph finished ##
    
    def run(self,b):
        with out:
            clear_output()
            
            generateConf(self.strfile,self.rskfile,self.usecmf,self.cmffile,self.cmffnc)
            
            if self.buildG == 'no': self.loadGraph()
            else: self.buildGraph()
            
            self.latList, self.lonList = [],[]
            for node in self.graph.nodes:
                self.latList.append(node.lat)
                self.lonList.append(node.lon)

            window = Tk()
            gui = GUI(window,self.mpfile,self.mpcoords,self.graph,self.latList,self.lonList)  
            # Start the window's event-loop
            window.mainloop() 


init = main()
widgets.interact(init.defStreets,Street_file='/datasets/streets/Zurich_w1_graphPoints.txt')
widgets.interact(init.defRisks,Risk_file='/datasets/bicyclerisk/Zurich_w1_RoadNodeRisk.csv')
widgets.interact(init.useComfFunct,useComfnc='yes')
widgets.interact(init.defComfort,Discomfort_file='')
#widgets.interact(init.defComfortFnc,Discomfort_function='code/comfortCalc.py')
widgets.interact(init.graphBuild,Build_graph='no')
widgets.interact(init.defGraph,Graph_file='/code/graph_files/WeightedGraph1.txt')
widgets.interact(init.defMap,Map_file='/img/map.gif')
widgets.interact(init.defMcoord,Map_coordinates=str([8.5141,8.5523,47.3650,47.3886]))

button = widgets.Button(description="Run")
display(button)

out = widgets.Output()
display(out)

button.on_click(init.run)

interactive(children=(Text(value='/datasets/streets/Zurich_w1_graphPoints.txt', description='Street_file'), Ou…

interactive(children=(Text(value='/datasets/bicyclerisk/Zurich_w1_RoadNodeRisk.csv', description='Risk_file'),…

interactive(children=(Text(value='yes', description='useComfnc'), Output()), _dom_classes=('widget-interact',)…

interactive(children=(Text(value='', description='Discomfort_file'), Output()), _dom_classes=('widget-interact…

interactive(children=(Text(value='no', description='Build_graph'), Output()), _dom_classes=('widget-interact',…

interactive(children=(Text(value='/code/graph_files/WeightedGraph1.txt', description='Graph_file'), Output()),…

interactive(children=(Text(value='/img/map.gif', description='Map_file'), Output()), _dom_classes=('widget-int…

interactive(children=(Text(value='[8.5141, 8.5523, 47.365, 47.3886]', description='Map_coordinates'), Output()…

Button(description='Run', style=ButtonStyle())

Output()